In [1]:
import Libs.Adb as adb
from Libs.Beholder import *
from Libs.StateMachine import StateMachine

In [2]:
adb.run_command(
    "shell am start com.jagex.runescape.android/com.jagex.android.MainActivity"
)

['Starting: Intent { act=android.intent.action.MAIN cat=[android.intent.category.LAUNCHER] cmp=com.jagex.runescape.android/com.jagex.android.MainActivity }']

In [3]:
a = Beholder(videoFrameGenerator=adb.ScreenCap())

a.addLayerModifer(Beholder_Layer_Chopper_Grayscale(from_layer="image", enabled=True))
a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="inventory_color",
        from_layer="image",
        x=1903,
        y=355,
        w=2222 - 1903,
        h=905 - 355,
        enabled=True,
    )
)
a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="inventory",
        from_layer="image_gray",
        x=1903,
        y=355,
        w=2222 - 1903,
        h=905 - 355,
        enabled=True,
    )
)
a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="Horizon",
        from_layer="image",
        x=690,
        y=320,
        w=1870 - 690,
        h=1,
        enabled=False,
    )
)

a.addLayerModifer(
    Beholder_Layer_Chopper_InRange(
        name="Horizon Enemy Finder",
        from_layer="Horizon",
        lbounds=[3, 0, 0],
        ubounds=[13, 248, 255],
        enabled=False,
    )
)

a.addLayerModifer(
    Beholder_Layer_Chopper_PeakExtractor(
        name="Horizon Enemy Finder Peaks",
        from_layer="Horizon Enemy Finder",
        threshhold=5,
        avg_fwd=5,
        enabled=False,
    )
)
adb.run_command("wait-for-device")

[]

In [4]:
raise Exception()

Exception: 

# Auto-Repeat smithing


In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="Button",
        layer="image_gray",
        filename="./rs3_templates/ui/Begin_project.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="UnfinishedSmithing",
        layer="inventory",
        filename="./rs3_templates/inventory/forge_item.png",
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="FinishedArrow",
        layer="image_gray",
        filename="./rs3_templates/ui/FinishedArrow.png",
        convertToGray=True,
    )
)

time_since_heat = time.time()
time_smithing = time.time()

In [ ]:
Button_count = 0
Anvil_count = 0
tts("starting autosmith")
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        if "UnfinishedSmithing" in matches:
            Button_count = 0
            Anvil_count = 0
            if time.time() - time_since_heat > 30:
                adb.run_command(f"shell input tap 1192 150")
                # tts("heat", volume=0.1)
                sleep(5)
                adb.run_command(f"shell input tap 1550 525")
                time_since_heat = time.time()
            sleep(7)
            continue
        if "FinishedArrow" in matches:
            center = matches["FinishedArrow"][0].center
            adb.run_command(f"shell input tap {center[0]} {center[1]}")
            tts(f"Completed in {int(time.time()-time_smithing)} seconds")
            time_smithing = time.time()
            sleep(1)
            continue
        if "Button" in matches:
            if Button_count == 0:
                center = matches["Button"][0].center
                adb.run_command(f"shell input tap {center[0]} {center[1]}")
                tts("starting item")
                Button_count += 1
            elif Button_count > 5:
                tts("Cannot continue smithing. Possible Out of ores")
                a.layers["image"].show()
                break
            else:
                tts("Finished, Please select new item you have 20 seconds")
                time.sleep(20)

                center = matches["Button"][0].center
                adb.run_command(f"shell input tap {center[0]} {center[1]}")
                Button_count += 1

    else:
        if Anvil_count == 0:
            adb.run_command(f"shell input tap 1550 525")
            time_since_heat = 0
            Anvil_count += 1
        else:
            tts("Cannot continue smithing. Possible Inventory Scanning Issue")
            a.layers["image"].show()
            break
    sleep(3.75)
tts("Cannot Continue")

In [ ]:
tts("finish autosmith")
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        if "UnfinishedSmithing" not in matches:
            break
    sleep(3)
tts("finished")

# Auto Smelt Ores

In [6]:
a.matchers = {}

a.addMatcher(
    Beholder_Image_Matcher(
        name="close_btb",
        layer="image_gray",
        filename="./rs3_templates/ui/crafting_dialog_close_button.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="Button",
        layer="image_gray",
        filename="./rs3_templates/ui/Begin_project.png",
        convertToGray=True,
    )
)

In [11]:
tts("starting auto smelt ")
s = time.time()
Button_count = 0
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        if "close_btb" in matches:
            Button_count = 0
            sleep(5)
            continue
        else:
            if "Button" in matches:
                if Button_count == 0:
                    center = matches["Button"][0].center
                    adb.run_command(f"shell input tap {center[0]} {center[1]}")
                    Button_count += 1
                    s = time.time()
                if Button_count == 2:
                    tts("Out of Ore Please select new ore type")
                    a.layers["image"].show()
                    sleep(15)
                if Button_count == 3:
                    break
                else:
                    adb.run_command(f"shell input tap 518 1029")
                    Button_count += 1
    else:
        tts(f"Fin, {int(time.time()-s)} Seconds.")
        adb.run_command(f"shell input tap 1150 780")
        sleep(3)
        adb.run_command(f"shell input tap 518 1029")
    sleep(3)
tts("Finished")

16:52:10.352192 - tts: starting auto smelt 
16:52:12.651066 - tts: Fin, 0 Seconds.
16:52:21.774805 - tts: Fin, 10 Seconds.


KeyboardInterrupt: 

# Bolt Featherer


In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="bolts",
        layer="gray",
        filename="./templates/inventory/addy_bolts.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="fletch",
        layer="gray",
        filename="./templates/ui/fletch.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="close_btb",
        layer="gray",
        filename="./templates/ui/crafting_dialog_close_button.png",
        convertToGray=True,
    )
)

in_menu = False
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        if "close_btb" in matches:
            print("Currently Crafting")
            sleep(2)
            continue
        if in_menu:
            if "fletch" in matches:
                center = matches["fletch"][0].center
                adb(f"shell input tap {center[0]} {center[1]}")
                in_menu = False
        else:
            if "bolts" in matches:
                center = matches["bolts"][0].center
                adb(f"shell input tap {center[0]} {center[1]}")
                in_menu = True
    else:
        break
    sleep(2)
tts("Cannot Continue")

a.layers["image"].show()

# Mine and store in OreChest/NotePaper/Mining Stamina

In [12]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="paper",
        layer="image_gray",
        filename="./rs3_templates/inventory/notepaper.png",
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="ore",
        layer="image_gray",
        filename="./rs3_templates/inventory/ore/rune.png",
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="orebox", layer="image_gray", filename="./rs3_templates/inventory/box2.png"
    )
)

s = time.time()
ore_box_full = False
start_mining = 0
total_mined = 0
mine_at_least = 500
previous_ore_count = 0
log_ores = True
log_resets = True
say_method = False
reset_mining_timeout = 15

In [ ]:
if total_mined > 0:
    tts(f"resuming AutoMiner {total_mined} out of {mine_at_least}")
else:
    tts("starting AutoMiner")
while True:
    if time.time() - start_mining >= reset_mining_timeout:
        if log_resets:
            print("reset mining")
        adb.run_command(f"shell input tap 1215 830")
        start_mining = time.time()
    matches = a.findMatches()
    if len(matches.keys()) > 0:
        if "ore" in matches:
            if log_ores:
                if previous_ore_count != len(matches["ore"]):
                    print(f"{len(matches['ore'])} ores in inventory")
                    previous_ore_count = len(matches["ore"])
            if len(matches["ore"]) >= 10:
                total_mined += len(matches["ore"])
                tts(f"took {int(time.time() - s)} seconds, {total_mined} Total")
                s = time.time()

                if mine_at_least < total_mined:
                    tts(f"finished aquiring {mine_at_least} ore")
                    break
                if not ore_box_full:
                    if "orebox" in matches:
                        if say_method:
                            tts("oreboxing")
                        center = matches["orebox"][0].center
                        adb.run_command(f"shell input tap {center[0]} {center[1]}")
                        time.sleep(1)
                        matches = a.findMatches()
                        if "ore" in matches.keys():
                            ore_box_full = True
                            tts("orebox is full, starting to use paper")

                        else:
                            start_mining = 0
                            continue

                if "ore" in matches and len(matches["ore"]) > 1:
                    if say_method:
                        tts("papering")
                    center = matches["ore"][0].center
                    adb.run_command(f"shell input tap {center[0]} {center[1]}")
                    time.sleep(0.5)
                    if "paper" in matches:
                        center = matches["paper"][0].center
                        adb.run_command(f"shell input tap {center[0]} {center[1]}")
                        start_mining = 0
                        time.sleep(2)
                    else:
                        tts("no more paper")
                        break

    if start_mining != 0:
        sleep(10)

tts("Exiting")

a.layers["inventory"].show()

16:52:40.906720 - tts: starting AutoMiner
reset mining
6 ores in inventory
reset mining
8 ores in inventory
reset mining
10 ores in inventory
16:53:27.172899 - tts: took 46 seconds, 10 Total
reset mining
2 ores in inventory
reset mining
4 ores in inventory
reset mining
6 ores in inventory
reset mining
8 ores in inventory
reset mining
10 ores in inventory
16:55:04.114070 - tts: took 94 seconds, 20 Total
reset mining
2 ores in inventory
reset mining
4 ores in inventory
reset mining
6 ores in inventory
reset mining
8 ores in inventory
10 ores in inventory
16:56:27.987408 - tts: took 81 seconds, 30 Total
reset mining
reset mining
2 ores in inventory
reset mining
5 ores in inventory
reset mining
7 ores in inventory
reset mining
9 ores in inventory


# Scareb gather - wip

In [ ]:
a.matchers = {}
for i in range(0, 5):
    print(i)
    a.addMatcher(
        Beholder_Image_Matcher(
            name="bug",
            layer="gray",
            filename=f"./rs3_templates/bug{i}.png",
            convertToGray=True,
            threshhold=0.9,
        )
    )
while True:
    matches = a.findMatches()
    if len(matches) > 0:
        print(matches)
    print(".")
tts("Cannot Continue")

# Enemy Attacker

In [ ]:
a.layer_modifiers["Horizon"].enabled = True
a.layer_modifiers["Horizon Enemy Finder"].enabled = True
a.layer_modifiers["Horizon Enemy Finder Peaks"].enabled = True

In [ ]:
while True:
    a.readNextImage()
    a.digestImage()
    item = max(a.layers["Horizon Enemy Finder Peaks"], key=lambda e: e[1] - e[0])
    print(item)
    center = (int(item[1] / 2 + item[0] / 2), 360)
    print(center)
    adb(f"shell input tap {center[0]} {center[1]}")
    sleep(5)

In [ ]:
a.layers

# Trout Fisher

In [ ]:
a.matchers = {}


a.addMatcher(
    Beholder_Image_Matcher(
        name="fish",
        layer="inventory",
        filename="./rs3_templates/fishing/fish.png",
        convertToGray=True,
        threshhold=0.95,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="button",
        layer="image_gray",
        filename="./rs3_templates/ui/Next.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="paper",
        layer="inventory",
        filename="./rs3_templates/inventory/notepaper.png",
        convertToGray=True,
    )
)
paper_time = time.time()
paper_time_timeout = 30
fishing_started = time.time()
storage_mode = False
matches = a.findMatches()
print({x: len(matches[x]) for x in matches.keys()})

In [ ]:
while True:
    matches = a.findMatches()
    print({x: len(matches[x]) for x in matches.keys()})
    if not storage_mode:
        if "button" in matches:
            center = matches["button"][0].center
            adb.run_command(f"shell input tap {center[0]} {center[1]}")
            tts(f"Completed in {int(time.time()-fishing_started)} seconds")
            fishing_started = time.time()
            print("start storage")
            storage_mode = True
        else:
            sleep(10)
    else:
        persist = False
        if "paper" not in matches:
            tts("No paper")
            break
        for _ in range(4):
            if "fish" in matches:
                print("storeing paper")
                center = matches["paper"][0].center
                adb.run_command(f"shell input tap {center[0]} {center[1]}")
                time.sleep(1)
                center = matches["fish"][0].center
                adb.run_command(f"shell input tap {center[0]} {center[1]}")
                print({x: len(matches[x]) for x in matches.keys()})
                time.sleep(1)
                matches = a.findMatches()
            storage_mode = False
        time.sleep(0.7)
        adb.run_command(f"shell input tap 1150 780")
        time.sleep(0.7)

# Tree Cutter

In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="paper",
        layer="inventory",
        filename="./rs3_templates/inventory/notepaper.png",
        convertToGray=True,
    )
)
a.addMatcher(
    Beholder_Image_Matcher(
        name="logs",
        layer="inventory",
        filename="./rs3_templates/inventory/logs/Achey.png",
        convertToGray=True,
    )
)
tree_locations = {
    "up": False,
    "right": False,
    "down": False,
    "left": True,
}
click_loc_pos = {
    "up": "1192 150",
    "right": "1550 525",
    "down": "1192 830",
    "left": "755 525",
}
tree_regrow_timer = 3
start_loggin = time.time()

In [ ]:
tree_regrow_timer_each = tree_regrow_timer / sum(
    [1 if tree_locations[a] else 0 for a in tree_locations]
)
tree_regrow_timer_each

In [ ]:
while True:
    for loc in ["up", "right", "down", "left"]:
        if tree_locations[loc]:
            adb.run_command(f"shell input tap {click_loc_pos[loc]}")
            sleep(tree_regrow_timer_each)
    matches = a.findMatches()
    if "logs" in matches:
        if len(matches["logs"]) >= 10:
            tts(f"cten {int(time.time()-start_loggin)}")
            start_loggin = time.time()
            center = matches["paper"][0].center
            adb.run_command(f"shell input tap {center[0]} {center[1]}")
            time.sleep(1)
            center = matches["logs"][0].center
            adb.run_command(f"shell input tap {center[0]} {center[1]}")
    sleep(2)

# repeat crafting

In [ ]:
a.matchers = {}
a.addMatcher(
    Beholder_Image_Matcher(
        name="close_btb",
        layer="image_gray",
        filename="./rs3_templates/ui/crafting_dialog_close_button2.png",
    )
)
start = time.time()
in_menu = False
auto_reset_timer = 180
min_cutoff_kill_timeout = 8

In [ ]:
while True:
    matches = a.findMatches()
    if (
        "close_btb" in matches
        and not in_menu
        and not time.time() - start > auto_reset_timer
    ):
        print(f"Currently Processsing { time.time() - start}")
        sleep(5)
        continue
    else:
        if in_menu:
            adb.run_command(f"shell input tap 1560 977")
            in_menu = False
            sleep(2.5)
        else:
            adb.run_command(f"shell input tap 1936 386")
            start = time.time()
            in_menu = True
            sleep(0.5)
tts("Cannot Continue")

a.layers["image"].show()

# Directional Clicker

In [ ]:
tree_locations = {
    "up": False,
    "right": True,
    "down": False,
    "left": True,
}
click_loc_pos = {
    "up": "1192 150",
    "right": "1550 525",
    "down": "1192 830",
    "left": "755 525",
}
click_delay = 5
while True:
    for loc in ["up", "right", "down", "left"]:
        if tree_locations[loc]:
            adb.run_command(f"shell input tap {click_loc_pos[loc]}")
            sleep(click_delay)

# Random Stuff

In [ ]:
s = time.time()

In [ ]:
time.time() - s

# Other Stuff

In [ ]:
time.time() - s

In [ ]:
447 / 20

In [ ]:
tts("Cannot Continue")

In [ ]:
a.layers

In [ ]:
a.layer_modifiers["Horizon"].enabled = True
a.layer_modifiers["Horizon Enemy Finder"].enabled = True

In [ ]:
target_color = (143, 4, 13)
tolarance = 75

target_info_cords = (460, 546)
target_info_color = (17, 42, 50)

In [ ]:
a.layers["Horizon Enemy Finder"].show()

In [ ]:
a.layers["Horizon Enemy Finder"].show()

In [ ]:
for item in a.matchers:
    print(item.name, item.data.shape)

In [ ]:
matches = a.findMatches()
print({x: len(matches[x]) for x in matches.keys()})

In [ ]:
matches["Salmon"][0].show(a)

In [ ]:
a.layers["gray"].data.shape